# Lab 4

Name: Nimish Kashyap

Reg: 20BRS1049

Date: 22/12/2022

In [ ]:
from keras import models
from keras import layers
from keras.datasets import mnist
import pandas as pd
from keras.utils import to_categorical

In [ ]:
(train_images, train_labels) , (test_images, test_labels) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
print(train_images.shape)
print(train_labels.shape)

print(test_images.shape)
print(test_labels.shape)



(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [ ]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
print(train_labels.shape)

(60000, 10)


In [ ]:
network = models.Sequential()
network.add(layers.Dense(512, activation="relu", input_shape=(28*28,)))
network.add(layers.Dense(10, activation="softmax"))

network.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
train_images = train_images.reshape((60000, 28*28))
train_images = train_images.astype('float32')/255

test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype('float32')/255


In [ ]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 9s 15ms/step - loss: 0.2539 - accuracy: 0.9267
Epoch 2/5
469/469 [==============================] - 6s 12ms/step - loss: 0.1041 - accuracy: 0.9692
Epoch 3/5
469/469 [==============================] - 6s 12ms/step - loss: 0.0676 - accuracy: 0.9795
Epoch 4/5
469/469 [==============================] - 6s 12ms/step - loss: 0.0495 - accuracy: 0.9850
Epoch 5/5
469/469 [==============================] - 6s 12ms/step - loss: 0.0380 - accuracy: 0.9886


In [ ]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

print(test_loss)

print(test_acc)

313/313 [==============================] - 1s 4ms/step - loss: 0.0666 - accuracy: 0.9799
0.06657201051712036
0.9799000024795532


## Implementing Deep Neural Network found on Github link.

In [ ]:
"""
network.py
~~~~~~~~~~
IT WORKS
A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        print(len(self.biases))
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        print(len(self.weights))

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""

        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {} : {} / {}".format(j,self.evaluate(test_data),n_test))
            else:
                print("Epoch {} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

##### Define the loader class to load data

Here a local copy of mnist data is used.

The data is loaded using **Pickle** API and returned as training data, validation data and test data respectively.

In [ ]:
import pickle
import gzip

# Third-party libraries
import numpy as np

def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.
    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.
    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.
    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.
    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('/content/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.
    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.
    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.
    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [ ]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data) 

##### Training the network using Stochastic Gradient Descent algorithm.

In [ ]:
network = Network([784, 30,10])
network.SGD(training_data, 30, 10, 3.0, test_data=test_data)

2
2


NameError: ignored

**_Conclusion_**:

Here we can see, 94.90% accuracy is obtained with neural network implemented. This network has been implemented using Stochastic Gradient Descent algorithm that minimizes the cost function. This algorithm implmented uses batching procedure for easier and faster processing of data.

Lab report 4

Change learning rate (very low very high)

Change number of neurons in hidden layer

Remove hidden layer and check accuracy

1. **Change** learning rate. 

In [ ]:
network = Network([784, 30,10])
network.SGD(training_data, 30, 10, 0.1, test_data=test_data)

Epoch 0 : 3536 / 10000
Epoch 1 : 5935 / 10000
Epoch 2 : 6856 / 10000
Epoch 3 : 7311 / 10000
Epoch 4 : 7602 / 10000
Epoch 5 : 7754 / 10000
Epoch 6 : 7872 / 10000
Epoch 7 : 8434 / 10000
Epoch 8 : 8587 / 10000
Epoch 9 : 8665 / 10000
Epoch 10 : 8744 / 10000
Epoch 11 : 8791 / 10000
Epoch 12 : 8827 / 10000
Epoch 13 : 8867 / 10000
Epoch 14 : 8876 / 10000
Epoch 15 : 8912 / 10000
Epoch 16 : 8925 / 10000
Epoch 17 : 8953 / 10000
Epoch 18 : 8976 / 10000
Epoch 19 : 8984 / 10000
Epoch 20 : 8990 / 10000
Epoch 21 : 9010 / 10000
Epoch 22 : 9011 / 10000
Epoch 23 : 9040 / 10000
Epoch 24 : 9052 / 10000
Epoch 25 : 9058 / 10000
Epoch 26 : 9068 / 10000
Epoch 27 : 9090 / 10000
Epoch 28 : 9104 / 10000
Epoch 29 : 9107 / 10000


**Conclusion**:
Reducing the learning rate has effectively reduced the accuracy.

In [ ]:
# INCREASING THE LEARNING RATE
network = Network([784, 30,10])
network.SGD(training_data, 30, 10, 10, test_data=test_data)

Epoch 0 : 8215 / 10000
Epoch 1 : 9075 / 10000
Epoch 2 : 9280 / 10000
Epoch 3 : 9337 / 10000
Epoch 4 : 9400 / 10000
Epoch 5 : 9374 / 10000
Epoch 6 : 9355 / 10000
Epoch 7 : 9373 / 10000
Epoch 8 : 9395 / 10000
Epoch 9 : 9410 / 10000
Epoch 10 : 9448 / 10000
Epoch 11 : 9359 / 10000
Epoch 12 : 9388 / 10000
Epoch 13 : 9443 / 10000
Epoch 14 : 9469 / 10000
Epoch 15 : 9468 / 10000
Epoch 16 : 9427 / 10000
Epoch 17 : 9483 / 10000
Epoch 18 : 9440 / 10000
Epoch 19 : 9460 / 10000
Epoch 20 : 9467 / 10000
Epoch 21 : 9481 / 10000
Epoch 22 : 9439 / 10000
Epoch 23 : 9462 / 10000
Epoch 24 : 9498 / 10000
Epoch 25 : 9509 / 10000
Epoch 26 : 9520 / 10000
Epoch 27 : 9502 / 10000
Epoch 28 : 9547 / 10000
Epoch 29 : 9510 / 10000


**Conclusion**:
Increasing the learning rate to 10 has increased the overall accuracy at the cost of higher training time of 5m 33s.

2. Changing the number of neurons in the hidden layer.


Here the number of neurons has been increased from 30 to 50.

In [ ]:
network = Network([784, 50,10])
network.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0 : 7466 / 10000
Epoch 1 : 9270 / 10000
Epoch 2 : 9357 / 10000
Epoch 3 : 9379 / 10000
Epoch 4 : 9462 / 10000
Epoch 5 : 9478 / 10000
Epoch 6 : 9513 / 10000
Epoch 7 : 9505 / 10000
Epoch 8 : 9519 / 10000
Epoch 9 : 9532 / 10000
Epoch 10 : 9561 / 10000
Epoch 11 : 9559 / 10000
Epoch 12 : 9556 / 10000
Epoch 13 : 9565 / 10000
Epoch 14 : 9569 / 10000
Epoch 15 : 9575 / 10000
Epoch 16 : 9569 / 10000
Epoch 17 : 9586 / 10000
Epoch 18 : 9570 / 10000
Epoch 19 : 9588 / 10000
Epoch 20 : 9578 / 10000
Epoch 21 : 9612 / 10000
Epoch 22 : 9587 / 10000
Epoch 23 : 9589 / 10000
Epoch 24 : 9589 / 10000
Epoch 25 : 9590 / 10000
Epoch 26 : 9589 / 10000
Epoch 27 : 9582 / 10000
Epoch 28 : 9605 / 10000
Epoch 29 : 9582 / 10000


**Conclusion**:

Here we can see that it has the time for the traininig to complete the epochs 
of about 11m 42s. But the effective accuracy has not been changed much. This signifies that increasing number of neurons does not necessarily means increased accuracy and we have to take other parameters into consideration.

3. Removing hidden layer completely.

In [ ]:
network = Network([784,10])
network.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0 : 6287 / 10000
Epoch 1 : 6443 / 10000
Epoch 2 : 6524 / 10000
Epoch 3 : 6510 / 10000
Epoch 4 : 6489 / 10000
Epoch 5 : 6674 / 10000
Epoch 6 : 7450 / 10000
Epoch 7 : 7404 / 10000
Epoch 8 : 7461 / 10000
Epoch 9 : 7580 / 10000
Epoch 10 : 8235 / 10000
Epoch 11 : 8275 / 10000
Epoch 12 : 8210 / 10000
Epoch 13 : 8285 / 10000
Epoch 14 : 8270 / 10000
Epoch 15 : 8250 / 10000
Epoch 16 : 8325 / 10000
Epoch 17 : 8307 / 10000
Epoch 18 : 9102 / 10000
Epoch 19 : 9152 / 10000
Epoch 20 : 9143 / 10000
Epoch 21 : 9177 / 10000
Epoch 22 : 9168 / 10000
Epoch 23 : 9149 / 10000
Epoch 24 : 9144 / 10000
Epoch 25 : 9150 / 10000
Epoch 26 : 9126 / 10000
Epoch 27 : 9162 / 10000
Epoch 28 : 9158 / 10000
Epoch 29 : 9159 / 10000


**Conclusion**:

Removing the hidden layer has resulted in lower accuracy as well as low absolute accuracy change on each epoch, but it has increased the training time significantly, about 1m 23s.

4. Adding another layer.

In [ ]:
network = Network([784, 50,20,10])
network.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0 : 9028 / 10000
Epoch 1 : 9271 / 10000
Epoch 2 : 9340 / 10000
Epoch 3 : 9382 / 10000
Epoch 4 : 9447 / 10000
Epoch 5 : 9441 / 10000
Epoch 6 : 9447 / 10000
Epoch 7 : 9480 / 10000
Epoch 8 : 9495 / 10000
Epoch 9 : 9487 / 10000
Epoch 10 : 9520 / 10000
Epoch 11 : 9550 / 10000
Epoch 12 : 9540 / 10000
Epoch 13 : 9538 / 10000
Epoch 14 : 9528 / 10000
Epoch 15 : 9547 / 10000
Epoch 16 : 9553 / 10000
Epoch 17 : 9511 / 10000
Epoch 18 : 9527 / 10000
Epoch 19 : 9515 / 10000
Epoch 20 : 9488 / 10000
Epoch 21 : 9519 / 10000
Epoch 22 : 9550 / 10000
Epoch 23 : 9509 / 10000
Epoch 24 : 9595 / 10000
Epoch 25 : 9612 / 10000
Epoch 26 : 9553 / 10000
Epoch 27 : 9523 / 10000
Epoch 28 : 9556 / 10000


**Conclusion**: 

Adding another layer with 20 neurons has increased the training time completely. But It does not have significance effect on the overall accuracy.
But the accuracy on first epoch was more than 90% and the change of accuracy was gradual.